ニューラルネットワークは本来「網」なので、要素であるニューロン（神経）が文字通り網の目状に絡まっています。ただそういう構造は複雑で作る事が難しいため、構造（モデル）を簡略化します。その一つのモデルがシーケンシャル（Sequential）モデルです。シーケンシャルというのは「線形の」という意味で、線形というのは線のように一列につながっているという事です。その繋がる元は「レイヤー(Layer)」と呼ばれているニューロン（パーセプトロン（Perseptron））の塊です：
　レイヤーの中にあるパーセプトロンが次のレイヤーに信号を送る。これをシーケンシャルに続けていき、最後のレイヤーまで届ける。この割と単純なモデルがシーケンシャルモデルです。Sequentialモデルにはこのレイヤーをいくらでも直列に追加する事が出来ます。

In [4]:
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
# データ取り込み
train = pd.read_csv("titanic_train.csv")
test = pd.read_csv("titanic_test.csv")
data_check = pd.read_csv("titanic_gender_submission.csv")



# 列削除
train_x = train.drop(["Survived", "Name", "Cabin", "Ticket", "PassengerId"], axis=1)
test_x = test.drop(["Name", "Cabin", "Ticket", "PassengerId"], axis=1)


# Survived列のみ抽出
train_y = train["Survived"]


# 欠損値を年齢の中央値。出港地は最多のSで補完
train_x["Age"] = train_x["Age"].fillna(train_x["Age"].median())
train_x["Embarked"] = train_x["Embarked"].fillna("S")

test_x["Age"] = test_x["Age"].fillna(test_x["Age"].median())
test_x["Fare"] = test_x["Fare"].fillna(test_x["Fare"].median())
test_x["Embarked"] = test_x["Embarked"].fillna("S")


# Sex、Embarked列をダミー変換
train_x_dummy = pd.get_dummies(train_x,columns=["Sex","Embarked"])
test_x_dummy = pd.get_dummies(test_x,columns=["Sex","Embarked"])


# 説明変数、目的変数分割
X_train, X_test, y_train, y_test = train_test_split(train_x_dummy, train_y, test_size=0.3, random_state=101)

In [15]:
import keras
# from keras.modelsというのはモデルが色々はいったモジュール。その中から「Sequentialを使う」というのがimport Sequential
from keras.models import Sequential
# レイヤーはkeras.layersにまとめられていて、その中のDenseを使う事を上で宣言
from keras.layers import Dense, Dropout


model = Sequential()

# Sequentialが持っているaddメソッドでレイヤーを追加
# addメソッドの引数にDenseレイヤーを与える
# Denseレイヤーも多様なパラメータで初期化できるのですが、最も単純な初期化が上のようにunitsに数値を与えるだけのもの
# unitsはDenseを作る時に唯一必須のパラメータ
#　Denseは全結合するので、上のSequentialモデルの各レイヤーのユニットは次のユニットにすべて繋がる
# 入力シェイプ（input_shape）　N次元のデータ
model.add(Dense(units = 16, input_shape = (X_train.shape[1], )))
model.add(Dense(units = 24, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(16, activation='relu'))

model.add(Dense(units = 1, activation='sigmoid'))


# 平均二乗誤差（Mean squared error）, 重み更新方法の変更　：　確率的勾配降下法（Stochastic gradient descent）
model.compile(loss = 'binary_crossentropy', optimizer = 'sgd', metrics=['accuracy'])

model.fit( X_train, y_train, epochs = 500, batch_size = 30 , verbose=0)


predictions = model.predict(X_test, batch_size = 5)
np.set_printoptions(suppress=True)
predictions
# np.set_printoptions(suppress=True)
# print( expect)

#getting predictions of test data
# list to series
# se = pd.Series(predictions)
# # creating new column of predictions in data_check dataframe
# data_check['check'] = se
# data_check['check'] = data_check['check'].str.get(0)
# data_check['check']
# 2値分類のうちどちらに分類されるか
y_pred_proba = model.predict(X_test) # [:, 1]
y_pred = np.where(y_pred_proba >0.5, 1, 0)
print("y_pred_proba", y_pred_proba)
print("y_pred", y_pred)
print('y_test', y_test.ravel())

y_pred_proba [[0.22249684]
 [0.84348696]
 [0.4856549 ]
 [0.6592986 ]
 [0.11499697]
 [0.06793502]
 [0.3865454 ]
 [0.10951206]
 [0.87240756]
 [0.6715486 ]
 [0.11499697]
 [0.08155477]
 [0.5007709 ]
 [0.10021895]
 [0.07270107]
 [0.05645806]
 [0.4358297 ]
 [0.14534202]
 [0.15788138]
 [0.722858  ]
 [0.07710218]
 [0.08815128]
 [0.6458997 ]
 [0.74108416]
 [0.06235513]
 [0.12529483]
 [0.15868685]
 [0.2648289 ]
 [0.48424566]
 [0.10643581]
 [0.6986976 ]
 [0.9846333 ]
 [0.64615256]
 [0.42931324]
 [0.15885356]
 [0.62695885]
 [0.08423546]
 [0.0656338 ]
 [0.24196115]
 [0.46811175]
 [0.62131745]
 [0.6715486 ]
 [0.58710676]
 [0.23437312]
 [0.10686663]
 [0.39205772]
 [0.05839589]
 [0.2070347 ]
 [0.07570773]
 [0.11499697]
 [0.62091446]
 [0.5786797 ]
 [0.7079229 ]
 [0.6666223 ]
 [0.64615256]
 [0.09134579]
 [0.14221746]
 [0.1390301 ]
 [0.2013556 ]
 [0.6936879 ]
 [0.8381541 ]
 [0.09150022]
 [0.05765837]
 [0.5408213 ]
 [0.06907737]
 [0.1634619 ]
 [0.7426943 ]
 [0.8055575 ]
 [0.6571011 ]
 [0.32429072]
 [0.368

In [9]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                176       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                408       
_________________________________________________________________
dropout (Dropout)            (None, 24)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                400       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 1,001
Trainable params: 1,001
Non-trainable params: 0
____________________________________________________

In [7]:
説明変数と目的変数に分割

import keras
from keras.utils.np_utils import to_categorical
# 説明変数と目的変数に分割
y_train = train["Survived"].values
 
COLUMNS = ["Pclass", "Sex", "Age", "SibSp", "Parch","Fare"]
x_train = train[COLUMNS].values
 
x_test = test[COLUMNS].values

お約束のようなコード。

モデルの作成


import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
 
# モデル作成 正直適当
model = Sequential()
model.add(Dense(32, input_shape=(len(COLUMNS),), activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
 
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
 
model.fit(
    x_train, 
    y_train, 
    epochs=30, 
    batch_size=1, 
    verbose=1)

SyntaxError: invalid character in identifier (<ipython-input-7-d5e935d25a2e>, line 13)

In [ ]:
from keras import models
from keras import layers
import tensorflow as tf
from sklearn import datasets
from keras.models import Sequential
from keras.layers import Dense, Activation
from matplotlib import pyplot

#訓練データとテストデータの分割
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(features_two, target, test_size=0.2)

#モデルの定義
model = models.Sequential()
model.add(layers.Dense(64, activation="relu", input_shape=(7, )))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1, activation="sigmoid"))

#モデルの構築
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

#学習の実行
history = model.fit(X_train.astype(np.float32),y_train,epochs=200)

#エポック毎のaccuracy結果グラフ化
pyplot.plot(history.history['accuracy'])
pyplot.title('model accuracy')
pyplot.ylabel('accuracy')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'test'], loc='upper left')
pyplot.show()

#評価の実行
score = model.evaluate(X_test.astype(np.float32),y_test,batch_size=1)
print(score[0])
print(score[1])

y_trainの答えが配列（マルチアウトプット）になっていますが、これはSequenceに加えた最後のDenseのユニット数と一緒でなければなりません。答えが例えば勝ち負けのようなカテゴリーであったり、何らかの平均値的な物である場合は単一の数になる事もあります（シングルアウトプット）。
